In [1]:
import pandas as pd

import torch

from discovery_utils import auto_circuit

from experiments import benchmark_experiment
from utils import get_data, load_gpt2_tl

torch.set_grad_enabled(False)

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

/home/jgcarrasco/.virtualenvs/mech_interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# acronyms -> 250
# ioi -> 150
# greater-than -> 250
n_patching = 250
n_val = 250
task = "greater-than"


In [3]:
threshold = 0.0853
include_mlps = True

rows = []

data = get_data(n_patching=n_patching, n_val=n_val, task=task)

patching_tokens = data["patching_tokens"].cuda()
patching_cache = data["patching_cache"]

val_tokens = data["val_tokens"].cuda()
val_answer_tokens = data["val_answer_tokens"].cuda()
val_logits = data["val_logits"]

gt_heads = data["gt_circuit"]

model = load_gpt2_tl()
circuit_heads, _ = auto_circuit(
    model, threshold, val_logits, val_tokens, patching_cache, 
    ablation_scheme="mean", include_mlps=include_mlps)

tpr = len([head for head in circuit_heads if head in gt_heads]) / len(gt_heads)
fpr = len([head for head in circuit_heads if head not in gt_heads]) / (144 - len(gt_heads))

rows.append([tpr, fpr])
df = pd.DataFrame(rows, columns=["TPR", "FPR"])

Loaded pretrained model gpt2-small into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
try:
    df_total = pd.read_csv(f"discovery_results/{task}_{threshold:.2f}{'_mlp' if include_mlps else ''}.csv")
    df_total = pd.concat([df_total, df])
except:
    df_total = df

df_total.to_csv(f"discovery_results/{task}_{threshold:.2f}{'_mlp' if include_mlps else ''}.csv", index=False)

In [5]:
df_total

,TPR,FPR
0,0.25,0.0
1,0.25,0.0
2,0.25,0.0
3,0.25,0.0
0,0.25,0.0


In [6]:
df_total.describe()

,TPR,FPR
count,5.00,5.0
mean,0.25,0.0
std,0.00,0.0
min,0.25,0.0
25%,0.25,0.0
50%,0.25,0.0
75%,0.25,0.0
max,0.25,0.0
